In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [6]:
df = pd.read_excel('kanye_lyrics.xlsx')
df.head()

,Album,Song,Year,Run-time,Featured artist(s),Writer(s),Skit,Lyrics
0,The College Dropout,Intro (Skit),2004,00:19:00,NaN,Kanye West,Y,"Kanye, can I talk to you for a minute? Me and ..."
1,The College Dropout,We Don't Care,2004,03:59:00,NaN,"Kanye West, Miri Ben-Ari, Ross Vannelli",N,"[Intro]\nOh yeah, I've got the perfect song fo..."
2,The College Dropout,Graduation Day,2004,01:22:00,NaN,"Kanye West, John Stephens, Miri Ben-Ari",N,What in the fuck was that Kanye?!\nI told you ...
3,The College Dropout,All Falls Down,2004,03:43:00,Syleena Johnson,"Kanye West, Lauryn Hill",N,"[Hook: Syleena Johnson + Kanye West]\nOh, when..."
4,The College Dropout,I'll Fly Away,2004,01:09:00,NaN,Albert E. Brumley,N,One glad morning\nWhen this life is over\nI'll...


In [20]:
lyrics = df.Lyrics
lyrics.head()

0    Kanye, can I talk to you for a minute? Me and ...
1    [Intro]\nOh yeah, I've got the perfect song fo...
2    What in the fuck was that Kanye?!\nI told you ...
3    [Hook: Syleena Johnson + Kanye West]\nOh, when...
4    One glad morning\nWhen this life is over\nI'll...
Name: Lyrics, dtype: object

In [88]:
text = ""
for lyric in lyrics:
    lyric = str(lyric)
    text = text + lyric
text = text.lower()
text = text.split(' ')

In [97]:
#chars = sorted(list(set(text)))
words = list(set(text))
words.sort()
words_to_int = dict((w, i) for i, w in enumerate(words))
# counter_list = dict(enumerate(words, 1))
# inv_map = {v: k for k, v in my_map.items()}

In [98]:
n_words = len(text)
n_vocab = len(words)
print("Total Characters: ", n_words)
print("Total Vocab: ", n_vocab)

Total Characters:  54547
Total Vocab:  12099


In [99]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]
	dataX.append([words_to_int[word] for word in seq_in])
	dataY.append(words_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  54447


In [100]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [101]:
#define model
# define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [102]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [103]:
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
54447/54447 [==============================] - 76s 1ms/step - loss: 7.8144

Epoch 00001: loss improved from inf to 7.81445, saving model to weights-improvement-01-7.8144.hdf5
Epoch 2/10
54447/54447 [==============================] - 71s 1ms/step - loss: 7.5707

Epoch 00002: loss improved from 7.81445 to 7.57070, saving model to weights-improvement-02-7.5707.hdf5
Epoch 3/10
22656/54447 [===========>..................] - ETA: 41s - loss: 7.4322

KeyboardInterrupt: 

In [104]:
filename = "weights-improvement-02-7.5707.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [109]:
int_to_word = dict(enumerate(words, 1))


In [120]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
print(start)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_word[value] for value in pattern]), "\"")
print('end seed')
output = ""
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    output = output + result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(output)
print("\nDone.")


14310
Seed:
" do
theynetsknockoffs,you
andthese,don'sknockoffs,yo?
justlightyh·a·m,do
theynetsknockoffs,you?"
but1:that’sbaddagiovanni,h·a·m,eventuallyseen
iunwanted
beautifulourselves

[hook]
(jesus9?
tellmovement
uh,scarykneltshaw]
howwars
so9?
telldrake]
unemploymentqueenturbomuthafuckin'lies,titties,step
zeta,king
withlast,niggerishlightyshaw]
howomen
ipatientlyeveryone,lightyi'llomen
iparks
itokenhenry
toothere"som...thing?
(ballshaw]
howdon'snecks,titties,know
nowherenetslessissues,go)

[hook:oh

[outro:okay
youokay
youokaywildest
younetsstoop
evenissues,now
whennetsstoop
evenissues,now
ohokay
youokaywildest
younetsstoop
evenissues,now
whennecks,titties,drizzy,issues,now
causeit)
no,drizzy,it

[hook:h·a·m,don'swanna...ninety-tworobocop
whenmovin'lighty9?
tellrobocop
she "
end seed
that’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat’sthat

In [121]:
model.predict("whats good")

AttributeError: 'str' object has no attribute 'ndim'